<a href="https://colab.research.google.com/github/inseungkang/learningalgos/blob/main/Controller_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drivedrive.mount('/content/drive', force_remount=True)

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Inseung_LearningAlgos/
!pip3 install c3d

import numpy as np
import pandas as pd
import os
import fnmatch
import c3d
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Inseung_LearningAlgos


In [ ]:
file_name = os.listdir()
print()

AB1_Session1_Right10_Left10.npz  AB3_Session3_Right10_Left8.npz
AB1_Session1_Right12_Left12.npz  AB3_Session3_Static.npz
AB1_Session1_Right14_Left14.npz  AB3_Session4_Right10_Left14.npz
AB1_Session1_Right6_Left6.npz	 AB3_Session4_Static.npz
AB1_Session1_Right8_Left8.npz	 AB3_Session5_Right10_Left12.npz
AB1_Session1_Static.npz		 AB3_Session5_Static.npz
AB1_Session2_Right10_Left8.npz	 AB4_Session1_Right10_Left10.npz
AB1_Session2_Static.npz		 AB4_Session1_Right12_Left12.npz
AB1_Session3_Right10_Left14.npz  AB4_Session1_Right14_Left14.npz
AB1_Session3_Static.npz		 AB4_Session1_Right6_Left6.npz
AB1_Session4_Right10_Left12.npz  AB4_Session1_Right8_Left8.npz
AB1_Session4_Static.npz		 AB4_Session1_Static.npz
AB1_Session5_Right10_Left6.npz	 AB4_Session2_Right12_Left10.npz
AB1_Session5_Static.npz		 AB4_Session2_Static.npz
AB2_Session1_Right10_Left10.npz  AB4_Session3_Right6_Left10.npz
AB2_Session1_Right12_Left12.npz  AB4_Session3_Static.npz
AB2_Session1_Right14_Left14.npz  AB4_Session4_Right14_L